In [95]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder



In [96]:
data_dir = '/home/rehan/catkin_ws/src/panda_research/hand_classifier/data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['push', 'pull', 'up', 'down', 'stop','rotate_cw','rotate_ccw']
# Iterate over each subdirectory (class folder) in the data directory
for i,class_name in enumerate    (class_names):
    
    class_path = os.path.join(data_dir, class_names[i])
    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            # count+=1
            if file_name.endswith('.npy'):
                  
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                # Check if the shape of the recorded array matches the expected shape
                expected_shape = (210, )  # Adjust the shape according to your data
                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    count+=1
                    recorded_array=[]
                else:
                    continue
    print(f'data points in class {class_name} are {count}') 
    count=0         
# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Step 2: Preprocess the data

# Perform any required preprocessing steps, such as scaling or normalization
# Here, we will use StandardScaler to scale the data
print(data.shape)
print (labels.shape)
print(len(class_names))
print(count)

data points in class push are 2000
data points in class pull are 2000
data points in class up are 2000
data points in class down are 2000
data points in class stop are 2000
data points in class rotate_cw are 2000
data points in class rotate_ccw are 2000
(14000, 210)
(14000,)
7
0


In [81]:
data_dir = '/home/rehan/catkin_ws/src/panda_research/hand_classifier/data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['push', 'pull', 'up', 'down', 'stop','rotate_cw','rotate_ccw']
class_counts = []  # Initialize a list to store the counts of valid samples per class

# Iterate over each subdirectory (class folder) in the data directory
for i, class_name in enumerate(class_names):
    class_path = os.path.join(data_dir, class_name)
    class_count = 0  # Initialize the count for the current class

    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)

            if file_name.endswith('.npy'):
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                expected_shape = (210,)  # Adjust the shape according to your data

                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    class_count += 1  # Increment the count for the current class

    class_counts.append(class_count)  # Store the count for the current class

# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Print the counts for each class
for i, class_name in enumerate(class_names):
    print(f"{class_name}: {class_counts[i]}")

push: 2000
pull: 2000
up: 2000
down: 2000
stop: 2000
rotate_cw: 2000
rotate_ccw: 2000


In [97]:
# Split the data and labels into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [102]:
scaler = StandardScaler()
normalizer = MinMaxScaler(feature_range=(0, 1))

data_scaled = scaler.fit_transform(data)
data_normalized = normalizer.fit_transform(data_scaled)

train_data, val_data, train_labels, val_labels = train_test_split(data_normalized, labels, test_size=0.2, random_state=42)

print(train_data.shape)
print(val_data.shape)
print(train_labels.shape)
print(val_labels.shape)

(11200, 210)
(2800, 210)
(11200,)
(2800,)


In [103]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(210,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))


In [104]:

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [105]:
# Train the model
model.fit(train_data, train_labels, batch_size=32, epochs=10, validation_data=(val_data, val_labels))

Epoch 1/10
350/350 [==============================] - 1s 1ms/step - loss: 1.1158 - accuracy: 0.6016 - val_loss: 0.1506 - val_accuracy: 0.9893
Epoch 2/10
350/350 [==============================] - 0s 1ms/step - loss: 0.3217 - accuracy: 0.8983 - val_loss: 0.0381 - val_accuracy: 0.9929
Epoch 3/10
350/350 [==============================] - 0s 1ms/step - loss: 0.2073 - accuracy: 0.9321 - val_loss: 0.0170 - val_accuracy: 0.9961
Epoch 4/10
350/350 [==============================] - 0s 1ms/step - loss: 0.1599 - accuracy: 0.9461 - val_loss: 0.0107 - val_accuracy: 0.9975
Epoch 5/10
350/350 [==============================] - 0s 1ms/step - loss: 0.1273 - accuracy: 0.9597 - val_loss: 0.0103 - val_accuracy: 0.9968
Epoch 6/10
350/350 [==============================] - 1s 1ms/step - loss: 0.1273 - accuracy: 0.9550 - val_loss: 0.0068 - val_accuracy: 0.9975
Epoch 7/10
350/350 [==============================] - 0s 1ms/step - loss: 0.1054 - accuracy: 0.9662 - val_loss: 0.0099 - val_accuracy: 0.9954
Epoch 

In [106]:
# Get the 9predicted labels from the model
predicted_labels = model.predict(val_data)
predictions = np.around(model.predict(val_data))
# print(predictions)
# Decode the predicted labels
print((predicted_labels.shape))
for i,prediction in enumerate (predictions):
    # print(np.argmax(predictions[i][0]))
    label=np.argmax(predictions)
    print(class_names[label])
# print(prediction_labels[0][np.argmax(prediction)])
print(class_names)

88/88 [==============================] - 0s 525us/step
(2800, 7)
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull
pull


In [90]:
import pickle
# Save the model
model.save('/home/rehan/catkin_ws/src/panda_research/hand_classifier/model12/keras_model.h5')

# Save the labels to a text file
with open('/home/rehan/catkin_ws/src/panda_research/hand_classifier/model12/labels.txt', 'w') as f:
    for label in class_names:
        f.write(label + '\n')
# Saving the scaler and normalizer objects
with open('/home/rehan/catkin_ws/src/panda_research/hand_classifier/model12/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

with open('/home/rehan/catkin_ws/src/panda_research/hand_classifier/model12/normalizer.pkl', 'wb') as normalizer_file:
    pickle.dump(normalizer, normalizer_file)